In [ ]:
'''CascadeClassifier를 이용한 얼굴 추출 (성능 안 좋음 -> HOG 추천)'''

import os
from PIL import Image
import cv2
import numpy as np

# 데이터 디렉토리 경로
data_dir = 'data'

# 새로운 데이터 디렉토리 경로
new_data_dir = 'new_data'
if not os.path.exists(new_data_dir):
    os.makedirs(new_data_dir)

# 감정 레이블 리스트
labels = ['anger']

for label in labels:
    # 각 감정 레이블 디렉토리 경로
    label_dir = os.path.join(data_dir, label)
    new_label_dir = os.path.join(new_data_dir, label)
    if not os.path.exists(new_label_dir):
        os.makedirs(new_label_dir)
    
    # 각 디렉토리 내 train, test, validation 디렉토리 경로
    for split in ['test']:
        split_dir = os.path.join(label_dir, split)
        new_split_dir = os.path.join(new_label_dir, split)
        if not os.path.exists(new_split_dir):
            os.makedirs(new_split_dir)
        
        # 각 디렉토리 내 이미지 파일 경로
        for filename in os.listdir(split_dir):
            file_path = os.path.join(split_dir, filename)
            new_file_path = os.path.join(new_split_dir, filename)
            
            try:
                # 이미지 로드
                image = cv2.imread(file_path)
                
                # 얼굴 검출
                face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
                faces = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(40, 40))
                
                if len(faces) > 0:
                    # 첫 번째 얼굴 영역 추출
                    x, y, w, h = faces[0]
                    
                    # 얼굴 영역 자르기
                    face_image = image[y:y+h, x:x+w]
                    
                    # 이미지 크기 조정
                    resized_image = cv2.resize(face_image, (224, 224))
                    
                    # 이미지 저장
                    cv2.imwrite(new_file_path, resized_image)
                else:
                    print(f"No face detected in {filename}")
                    continue
            except (cv2.error, OSError) as e:
                print(f"Error processing {filename}: {e}")
                continue


In [ ]:
'''HOG 방식을 이용한 얼굴 추출'''

#오픈 cv 설치 및 라이브러리 설치
import cv2
import imutils
import matplotlib.pyplot as plt
import dlib
import numpy as np


#이미지 전처리 및 HOG를 이용한 얼굴 추출
def image_processing(image):
    hog_face_detector = dlib.get_frontal_face_detector()
    face_detections = hog_face_detector(image)
    f=face_detections

    left,right,top,bottom = f[0].left(),f[0].right(),f[0].top(),f[0].bottom()
    cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0),2)

    test_img = image.copy()
    test_img_cut = test_img[top:bottom,left:right]

    image_resized = cv2.resize(test_img_cut, (245, 256),2)
    return image_resized


In [ ]:
''' 리사이징 '''
import os
import cv2

# 리사이징 함수
def resize_image(image, size=(512, 512)):
    return cv2.resize(image, size)

# 데이터셋 경로
base_dir = './new_data'
# 새로운 저장 경로
new_base_dir = './new_data_1'

# 감정별 디렉토리 목록
emotion_dirs = ['anger', 'happy']
# 새로운 감정별 디렉토리 목록
new_emotion_dirs = ['anger_1', 'happy_1']

# 서브 디렉토리 목록
subdirs = ['test', 'train', 'validation']

# 디렉토리 순회
for emotion_dir, new_emotion_dir in zip(emotion_dirs, new_emotion_dirs):
    for subdir in subdirs:
        data_dir = os.path.join(base_dir, emotion_dir, subdir)
        save_dir = os.path.join(new_base_dir, new_emotion_dir, subdir)
        
        # 저장할 디렉토리 생성
        os.makedirs(save_dir, exist_ok=True)
        
        # 이미지 파일 목록 가져오기
        image_files = [os.path.join(data_dir, img) for img in os.listdir(data_dir) if img.endswith(('.jpg', '.png', '.jpeg'))]
        
        # 이미지 리사이징 및 저장
        for i, img_path in enumerate(image_files):
            try:
                image = cv2.imread(img_path)
                if image is not None:
                    resized_image = resize_image(image)
                    save_path = os.path.join(save_dir, f'{emotion_dir}_{i+1}.jpg')
                    cv2.imwrite(save_path, resized_image)
                else:
                    print(f"Error loading image {img_path}")
            except Exception as e:
                print(f"Error processing image {img_path}: {str(e)}")

print("All images have been resized and saved to the new directories.")


In [ ]:
''' 전처리_크롭_리사이징 '''
import os
import cv2
import dlib

# 얼굴 추출 함수
def image_processing(image):
    hog_face_detector = dlib.get_frontal_face_detector()
    face_detections = hog_face_detector(image)
    if len(face_detections) == 0:
        return None  # 얼굴이 검출되지 않은 경우 None 반환

    f = face_detections[0]
    left, right, top, bottom = f.left(), f.right(), f.top(), f.bottom()

    if left < 0 or top < 0 or right > image.shape[1] or bottom > image.shape[0]:
        return None  # 얼굴 위치가 이미지 범위를 벗어나는 경우 None 반환

    test_img_cut = image[top:bottom, left:right]
    image_resized = cv2.resize(test_img_cut, (512, 512))
    return image_resized

# 데이터셋 경로
base_dir = './data/unrest'
save_base_dir = './new_data/unrest'

# 디렉토리 목록
subdirs = ['test', 'train', 'validation']

# 디렉토리 순회
for subdir in subdirs:
    data_dir = os.path.join(base_dir, subdir)
    save_dir = os.path.join(save_base_dir, subdir)
    
    # 저장할 디렉토리 생성
    os.makedirs(save_dir, exist_ok=True)
    
    # 이미지 파일 목록 가져오기
    image_files = [os.path.join(data_dir, img) for img in os.listdir(data_dir) if img.endswith(('.jpg', '.png', '.jpeg'))]
    
    # 이미지 처리 및 저장
    for i, img_path in enumerate(image_files):
        try:
            image = cv2.imread(img_path)
            if image is not None:
                processed_image = image_processing(image)
                if processed_image is not None:
                    save_path = os.path.join(save_dir, f'unrest_{i+1}.jpg')
                    cv2.imwrite(save_path, processed_image)
                else:
                    print(f"No face detected in {img_path}")
            else:
                print(f"Error loading image {img_path}")
        except Exception as e:
            print(f"Error processing image {img_path}: {str(e)}")


In [ ]:
'''gray scale로 변환'''
import os
import glob
from PIL import Image

# 원본 디렉토리와 목표 디렉토리 설정
base_dir = './new_data/'
target_dir = './gray_data/'

# 감정 디렉토리 리스트
emotions = ['anger', 'happy', 'hurt', 'neutral', 'panic', 'sad', 'unrest']

# 목표 디렉토리가 없는 경우 생성
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# 각 감정 디렉토리 안의 test, train, validation 디렉토리 안의 이미지를 그레이스케일로 변환하여 저장
for emotion in emotions:
    for split in ['test', 'train', 'validation']:
        source_path = os.path.join(base_dir, emotion, split)
        target_path = os.path.join(target_dir, emotion, split)

        # 목표 디렉토리가 없는 경우 생성
        if not os.path.exists(target_path):
            os.makedirs(target_path)

        # 모든 이미지 파일을 찾기
        image_files = glob.glob(source_path + '/*.jpg')
        for image_file in image_files:
            with Image.open(image_file) as img:
                gray_img = img.convert('L')
                
                # 원본 파일명 가져오기
                file_name = os.path.basename(image_file)
                
                # 새로운 디렉토리에 그레이스케일 이미지를 저장
                gray_img.save(os.path.join(target_path, file_name))

print("모든 이미지 파일이 그레이스케일로 변환되어 './gray_data' 디렉토리에 저장되었습니다.")


In [ ]:
'''데이터 증강(Papper&Salt, Brightness)'''
'''gray scale 변환 후 실행'''
import os
import cv2
import numpy as np
import random

# 데이터 디렉터리 경로
data_dir = './gray_data/Train'

# 증강 기법 함수
def add_noise(image, noise_type="gaussian", mean=0, var=0.1):
    if noise_type == "gaussian":
        row, col = image.shape
        sigma = var ** 0.5
        gauss = np.random.normal(mean, sigma, (row, col))
        noisy_image = image + gauss
    elif noise_type == "s&p":
        row, col = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i, int(num_salt)) for i in image.shape]
        out[coords[0], coords[1]] = 255

        # Pepper mode
        num_pepper = np.ceil(amount * image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i, int(num_pepper)) for i in image.shape]
        out[coords[0], coords[1]] = 0
        noisy_image = out
    return noisy_image

def adjust_brightness(image, brightness_factor):
    adjusted_image = cv2.convertScaleAbs(image, alpha=1, beta=brightness_factor)
    return adjusted_image

# 데이터 증강 함수
def augment_data(input_dir, output_dir, num_augmented_samples=5):
    for emotion_dir in os.listdir(input_dir):
        emotion_path = os.path.join(input_dir, emotion_dir)
        if os.path.isdir(emotion_path):
            output_emotion_dir = os.path.join(output_dir, emotion_dir)
            os.makedirs(output_emotion_dir, exist_ok=True)

            for filename in os.listdir(emotion_path):
                if filename.endswith(".jpg"):
                    image_path = os.path.join(emotion_path, filename)
                    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

                    # 잡음 추가 (Salt and Pepper Noise)
                    for i in range(num_augmented_samples):
                        noisy_image = add_noise(image, noise_type="s&p")
                        output_path = os.path.join(output_emotion_dir, f"{os.path.splitext(filename)[0]}_noise.jpg")
                        cv2.imwrite(output_path, noisy_image)

                    # 밝기 조정
                    for i in range(num_augmented_samples):
                        brightness_factor = random.uniform(-50, 50)
                        adjusted_image = adjust_brightness(image, brightness_factor)
                        output_path = os.path.join(output_emotion_dir, f"{os.path.splitext(filename)[0]}_brightness.jpg")
                        cv2.imwrite(output_path, adjusted_image)

# 데이터 증강 실행
augment_data(data_dir, 'augmented_data', num_augmented_samples=1)


In [15]:
'''전체 데이터셋 확인'''
import os

for dir in ['Test', 'Train', 'Validation']:
    directory = f"./gray_data2/{dir}"

    total_count = 0
    file_counts = {}

    for subdir in ["anger", "happy", "hurt", "neutral", "panic", "sad", "unrest"]:
        subdir_path = os.path.join(directory, subdir)
        file_count = len(os.listdir(subdir_path))
        file_counts[subdir] = file_count
        total_count += file_count

    print(f"{dir} : {file_counts}, Total: {total_count}")

    

Test : {'anger': 1004, 'happy': 1225, 'hurt': 1221, 'neutral': 645, 'panic': 1268, 'sad': 1266, 'unrest': 1210}, Total: 7839
Train : {'anger': 15132, 'happy': 15546, 'hurt': 15183, 'neutral': 16920, 'panic': 15333, 'sad': 15366, 'unrest': 15342}, Total: 108822
Validation : {'anger': 702, 'happy': 989, 'hurt': 1022, 'neutral': 1029, 'panic': 960, 'sad': 987, 'unrest': 981}, Total: 6670


In [4]:
'''데이터 라벨 생성'''
import os
import csv

def create_image_csv(root_dir, output_file):
    image_data = []
    
    # 루트 디렉토리 아래의 모든 하위 디렉토리 탐색
    for class_name in os.listdir(root_dir):
        class_dir = os.path.join(root_dir, class_name)
        
        # 디렉토리인 경우에만 처리
        if os.path.isdir(class_dir):
            # 각 클래스 디렉토리 내의 이미지 파일 탐색
            for image_file in os.listdir(class_dir):
                # 이미지 파일 확장자 확인 (필요한 경우 더 추가)
                if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                    image_path = os.path.join(class_dir, image_file)
                    image_data.append([image_path, class_name])
    
    # CSV 파일 작성
    with open(output_file, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['image_path', 'class'])  # 헤더 작성
        csv_writer.writerows(image_data)

root_directory = '/home/yrok/JHK/gray_data/train'
output_csv = 'train_data.csv'

create_image_csv(root_directory, output_csv)
print(f"CSV 파일이 생성되었습니다: {output_csv}")

In [3]:
'''전체 데이터셋 확인'''
import os

for dir in ['test', 'train', 'valid']:
    directory = f"./gray_data/{dir}"

    total_count = 0
    file_counts = {}

    for subdir in ["anger", "happy", "neutral", "panic", "sad"]:
        subdir_path = os.path.join(directory, subdir)
        file_count = len(os.listdir(subdir_path))
        file_counts[subdir] = file_count
        total_count += file_count

    print(f"{dir} : {file_counts}, Total: {total_count}")

    

test : {'anger': 1004, 'happy': 1225, 'neutral': 645, 'panic': 1268, 'sad': 1266}, Total: 5408
train : {'anger': 5044, 'happy': 5182, 'neutral': 5640, 'panic': 5111, 'sad': 5122}, Total: 26099
valid : {'anger': 702, 'happy': 989, 'neutral': 1029, 'panic': 960, 'sad': 987}, Total: 4667
